## Load data

In [1]:
import pandas as pd
import ast
import numpy as np

# Load in data
admissions = 'tedsa_puf_2019.csv'
df_raw = pd.read_csv(f'../../Downloads/{admissions}')

## Filter out select rows and columns

In [2]:
# Get count of original number of rows
old_rows = len(df_raw)

# Drop defined columns (year of admission, case id, geographic metro area, geographic division, geographic region)
# columns_to_drop = ['ADMYR', 'CASEID', 'CBSA2010', 'DIVISION', 'REGION']
columns_to_drop = ['ADMYR', 'CASEID', 'CBSA2010']
df = df_raw.drop(columns=columns_to_drop)
print(f'Dropped {len(columns_to_drop)} columns ({len(df.columns)} remain)')

# Drop values where dependent variable is unknown
df = df[df['METHUSE'] != -9]

# Only keep patients admitted with self-described use of an opioid as their primary substance use (i.e., SUB1 = 5, 6, or 7)
df = df[df['SUB1'].between(5, 7)]
new_rows = len(df)
percent_change = round(100*(old_rows-new_rows)/old_rows, 1)
print(f'Dropped {"{:,}".format(old_rows-new_rows)} observations or {percent_change}% of the data ({"{:,}".format(new_rows)} rows remain)')

df = df.reset_index(drop='index')

Dropped 3 columns (59 remain)
Dropped 1,340,233 observations or 71.9% of the data (524,134 rows remain)


## Make dataset human-readable

In [3]:
# Load in variable dictionary
with open('VariableDictionary.txt') as file:
    variable_dict_string = file.read()
    variable_dict = ast.literal_eval(variable_dict_string)

# Rename entries in column according to dictionary
df2 = df.copy()
for col, col_dict in variable_dict.items():
    if col in df2.columns:
        for old_value, new_value in variable_dict[col].items():
            df2[col] = df2[col].replace(old_value, new_value)

# Rename "-9" values as "Unknown"
for col in df2.columns:
    df2[col] = df2[col].replace(-9, 'Unknown')

# Merge DETNLF (detailed not in labor force) into EMPLOY==4 (not in labor force)
detailed_employ = []

for idx, value in df2.iterrows():
    if value['EMPLOY'] == 'NotInLaborForce':
        if value['DETNLF'] == 'Unknown':
            # Assign 'UnknownNotInLaborForce' if 'NotInLaborForce' and 'Unknown'
            detailed_employ.append('UnknownNotInLaborForce')
        else:
            # Otherwise, assign as the DETNLF value
            detailed_employ.append(value['DETNLF'])
    else:
        # Assign the EMPLOY value if not 'NotInLaborForce'
        detailed_employ.append(value['EMPLOY'])

# Add a new column for detailed employment and drop the two source columns
df2['DETEMPLOY'] = detailed_employ
df2 = df2.drop(columns=['EMPLOY', 'DETNLF'])

## Percent receiving MOUD by variable subgroup

In [4]:
# Calculate the treatment rate for each subgroup
total = len(df2)
df3 = df2.copy()
df3 = df3.replace({'MethUse':1, 'NoMethUse':0})

# Loop through each variable, append frequencies to a running dataframe
df_freq = pd.DataFrame()
for col in df3.columns:

    # Calculate treatment rates
    df_mean = df3.groupby([col])['METHUSE'].mean()
    df_mean = pd.DataFrame(df_mean)

    # Calculate admission counts
    df_count = df3.groupby([col])['METHUSE'].count()
    df_count = pd.DataFrame(df_count)

    # Combine, rename columns
    df_running = pd.merge(df_mean, df_count, how='inner', left_index=True, right_index=True)
    df_running = df_running.reset_index()
    df_running = df_running.rename(columns={'METHUSE_x': 'percent_receiving_moud', 'METHUSE_y': 'total_admissions', col: 'subgroup'})

    # Calculate share of admissions, reorder columns, add group
    df_running['group_share_of_admissions'] = df_running['total_admissions']/total
    df_running = df_running[['subgroup', 'total_admissions', 'group_share_of_admissions', 'percent_receiving_moud']]
    df_running.insert(0, 'group', col)

    # Append results before moving to next columns
    df_freq = pd.concat([df_freq, df_running])

# Show results
df_freq = df_freq.reset_index(drop=True)
df_freq

,group,subgroup,total_admissions,group_share_of_admissions,percent_receiving_moud
0,STFIPS,AK,1603,0.003058,0.492826
1,STFIPS,AL,5774,0.011016,0.297194
2,STFIPS,AR,1947,0.003715,0.337956
3,STFIPS,AZ,10189,0.019440,0.181176
4,STFIPS,CA,42031,0.080191,0.587804
...,...,...,...,...,...
349,DETEMPLOY,RetiredOrDisabled,38813,0.074052,0.489759
350,DETEMPLOY,Student,1784,0.003404,0.314462
351,DETEMPLOY,Unemployed,191417,0.365206,0.372052
352,DETEMPLOY,Unknown,23056,0.043989,0.573603


In [5]:
# Uncomment to save results
# df_freq.to_csv('percent_receiving_moud.csv', index=False)

# Describe MOUD treatment frequency for each variable's subgroup and living status
Calculate the percent difference between the homeless and housed as well as its statistical difference

In [44]:
# Test out calculate treatment difference for each group
import pandas as pd
from scipy.stats import ttest_ind

# Simplify dataset
df_ttest = df2.copy()
df_ttest = df_ttest[df_ttest['LIVARAG'] != 'Unknown']
# df_ttest = df_ttest[df_ttest['LIVARAG'] != 'DependLiving']
df_ttest['LIVARAG'] = df_ttest['LIVARAG'].replace({'DependLiving':'Housed', 'IndependentLiving':'Housed'})
df_ttest['METHUSE'] = df_ttest['METHUSE'].replace({'MethUse':1, 'NoMethUse':0})


df_all = pd.DataFrame(columns=['HomelessAdmissions', 'HousedAdmissions', 'HomelessMOUD', 'HousedMOUD', 'PercentDifference', 'TStat', 'PValue', 'Significance'])

df_columns = [c for c in df_ttest.columns if c != 'LIVARAG']
for col in df_columns:
    # Create loop assets
    col_array = df_ttest[col].sort_values().unique()
    col_list = col_array.tolist()
    df_col = pd.DataFrame(columns=['Group', 'HomelessAdmissions', 'HousedAdmissions', 'HomelessMOUD', 'HousedMOUD', 'PercentDifference', 'TStat', 'PValue', 'Significance'])

    # Loop through each subgroup within each column
    for s in col_list:
        dft_temp = df_ttest[df_ttest[col] == s]

        # Group by living arrangement and MOUD, then store those variables for later (if they exist)
        dft_count = dft_temp.groupby('LIVARAG')['METHUSE'].count()
        housed_count = dft_count.loc['Housed']
        try:
            homeless_count = dft_count.loc['Homeless']
        except KeyError:
            homeless_count = 0

        dft_mean = dft_temp.groupby('LIVARAG')['METHUSE'].mean()
        housed_moud = dft_mean.loc['Housed']
        try:
            homeless_moud = dft_mean.loc['Homeless']
        except KeyError:
            homeless_moud = np.nan

        # Calculate the percent difference (if there is a homeless group in that state)
        if housed_moud > 0:
           percent_difference = (homeless_moud - housed_moud)/housed_moud
        # elif homeless_moud = 0:
        #     percent_difference
        else:
            percent_difference = np.nan

        # Perform a t-test between the two groups
        group_A_values = dft_temp[dft_temp['LIVARAG'] == 'Homeless']['METHUSE']
        group_B_values = dft_temp[dft_temp['LIVARAG'] == 'Housed']['METHUSE']
        t_stat, p_value = ttest_ind(group_A_values, group_B_values)

        # Add significance based on p-value
        if p_value < 0.001:
            significance = '****'
        elif p_value < 0.01:
            significance = '***'
        elif p_value < 0.05:
            significance = '**'
        elif p_value <0.1:
            significance = '*'
        else:
            significance = ''

        # Add values to a dictionary and round
        dict_results = {'HomelessAdmissions':homeless_count, 'HousedAdmissions':housed_count, 'HomelessMOUD':homeless_moud, 'HousedMOUD':housed_moud, 'PercentDifference':percent_difference, 'TStat':t_stat, 'PValue':p_value}
        for key, value in dict_results.items():
            dict_results[key] = round(value, 6)

        # Add significance and column group to dictionary, then add dictionary to dataframe
        dict_results['Significance'] = significance
        dict_results['Group'] = col
        df_col.loc[s] = dict_results

    # Add column df to running aggregate
    df_all = pd.concat([df_all, df_col])

# Clean up final column
# df_all = df_all.drop_duplicates()
df_all = df_all.reset_index().rename(columns={'index': 'Subgroup'})
df_all = df_all[['Group', 'Subgroup', 'HomelessAdmissions', 'HousedAdmissions', 'HomelessMOUD', 'HousedMOUD', 'PercentDifference', 'TStat', 'PValue', 'Significance']]

/Users/mymac/PycharmProjects/TEDSA/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/mymac/PycharmProjects/TEDSA/venv/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [53]:
# Merge dataframes together
df_combo = pd.merge(df_freq, df_all, left_on=['group', 'subgroup'], right_on=['Group', 'Subgroup'], how='outer')
df_combo = df_combo.drop(columns=['Group', 'Subgroup'])

# df_combo.to_csv('frequencies.csv') #uncomment to save file
df_combo

,group,subgroup,total_admissions,group_share_of_admissions,percent_receiving_moud,HomelessAdmissions,HousedAdmissions,HomelessMOUD,HousedMOUD,PercentDifference,TStat,PValue,Significance
0,STFIPS,AK,1603,0.003058,0.492826,333,1214,0.387387,0.518122,-0.252324,-4.249484,0.000023,****
1,STFIPS,AL,5774,0.011016,0.297194,379,5170,0.118734,0.312766,-0.620376,-8.004596,0.000000,****
2,STFIPS,AR,1947,0.003715,0.337956,203,1741,0.103448,0.364159,-0.715925,-7.541382,0.000000,****
3,STFIPS,AZ,10189,0.019440,0.181176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,STFIPS,CA,42031,0.080191,0.587804,10568,31443,0.344436,0.669338,-0.485408,-61.263606,0.000000,****
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,DETEMPLOY,RetiredOrDisabled,38813,0.074052,0.489759,5653,32425,0.340881,0.513462,-0.336112,-24.136788,0.000000,****
350,DETEMPLOY,Student,1784,0.003404,0.314462,110,1578,0.254545,0.322560,-0.210859,-1.480946,0.138808,
351,DETEMPLOY,Unemployed,191417,0.365206,0.372052,35629,148939,0.287210,0.384748,-0.253512,-34.445064,0.000000,****
352,DETEMPLOY,Unknown,23056,0.043989,0.573603,1338,15698,0.325859,0.708307,-0.539946,-29.468817,0.000000,****
